In [347]:
#解析生成json的内容
#解析决策过程的日志
import os
import json
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def paese_result_json(log_path):
    #读取json文件
    res=json.load(open(log_path,"r"))['update']
    #去除掉init_state部分
    static={}
    
    for r in res:
        actions,states,tick=r['action'],r['state'],r['tick']
        
        for action in actions:
            if str(action['class']) + '_' + str(action['id']) not in static:
                static[str(action['class']) + '_' + str(action['id'])]={}
            static[str(action['class']) + '_' + str(action['id'])][action['action_type']]=static[str(action['class']) + '_' + str(action['id'])][action['action_type']]+1 if action['action_type'] in static[str(action['class']) + '_' + str(action['id'])] else 1
            if 'damage' not in static[str(action['class']) + '_' + str(action['id'])]:
                static[str(action['class']) + '_' + str(action['id'])]['damage']=[]
            if 'damage' in action:
                for damage in action['damage']:
                    static[str(action['class']) + '_' + str(action['id'])]['damage'].append(damage[1][0])
                    
                    
        for state in states:
            if len(state)>0:
                for change in state:
                    if change[1][2]=="Hp":
                        #pass
                        if str(change[1][0]) + '_' + str(change[1][1]) not in static:
                            static[str(change[1][0]) + '_' + str(change[1][1])]={}
                        
                        if '最终HP' not in static[str(change[1][0]) + '_' + str(change[1][1])]:
                            static[str(change[1][0]) + '_' + str(change[1][1])]['最终HP']=''
                        static[str(change[1][0]) + '_' + str(change[1][1])]['最终HP']=change[2][1]
            
    for k in static.keys():
        static[k]['总伤害']=sum(static[k]['damage'])
        static[k]['总伤害次数']=len(static[k]['damage'])
        static[k]['平均伤害']=sum(static[k]['damage'])/len(static[k]['damage']) if len(static[k]['damage'])>0 else 0
        if 'RIGHT' not in static[k]:
            static[k]['RIGHT']=0
        if 'LEFT' not in static[k]:
            static[k]['LEFT']=0
        if 'TOP' not in static[k]:
            static[k]['TOP']=0
        if 'BOTTOM' not in static[k]:
            static[k]['BOTTOM']=0
        static[k]['总移动次数']=static[k]['RIGHT']+static[k]['LEFT']+static[k]['TOP']+static[k]['BOTTOM']


        #隐藏具体伤害
        #static[k].pop('damage')
        #按照key 排序
        static[k]=dict(sorted(static[k].items(),key=lambda x:x[0]))
    return static


In [357]:
log_files_path = os.getcwd()+"/test_data"
    #读取这个路径下所有文件
log_files = os.listdir(log_files_path)
for log_file in log_files:
    if log_file.endswith(".json"):
        log_path = log_files_path+"/"+log_file
        static_res_json=paese_result_json(log_path)
        df=pd.DataFrame(static_res_json).T

        print(log_file)
        display(df)

for_qiangye_skill_99.json


,BOTTOM,LEFT,RIGHT,SKILL_77,TOP,WAIT,damage,平均伤害,总伤害,总伤害次数,总移动次数,最终HP
monster_18714666,4,5,4,8,6,13,"[94.5, 75.0, 97.5, 75.0, 75.0, 75.0, 75.0, 96.0]",82.875,663.0,8,19,NaN
hero_5002,1,2,17,NaN,16,5,[],0,0,0,36,337.0


for_qiangye_skill_82.json


,BOTTOM,LEFT,RIGHT,SKILL_77,TOP,WAIT,damage,平均伤害,总伤害,总伤害次数,总移动次数,最终HP
monster_18714666,5,4,17,8,5,9,"[75.0, 0, 75.0, 0, 82.5, 100.0, 75.0, 0, 75.0, 100.0, 75.0, 0, 75.0, 100.0, 110.25, 100.0]",65.171875,1042.75,16,31,200.0
hero_5002,0,3,2,NaN,15,8,[],0,0,0,20,357.25


for_qiangye_skill_85.json


,BOTTOM,LEFT,RIGHT,SKILL_77,TOP,WAIT,damage,平均伤害,总伤害,总伤害次数,总移动次数,EFFECT_64,最终HP
monster_18714666,7,6,8,12,9,8,"[75.0, 75.0, 75.0, 96.75, 75.0, 75.0, 75.0, 105.75, 75.0, 103.5, 96.0, 75.0]",83.5,1002.0,12,30,NaN,NaN
hero_5002,1,0,4,NaN,10,10,[],0,0,0,15,13,648.0


for_qiangye_skill_81.json


,BOTTOM,LEFT,RIGHT,SKILL_77,TOP,WAIT,damage,平均伤害,总伤害,总伤害次数,总移动次数,最终HP
monster_18714666,4,5,4,8,6,13,"[75.0, 61.25, 103.5, 90.0, 103.5, 75.0, 75.0, 75.0]",82.28125,658.25,8,19,NaN
hero_5002,1,2,17,NaN,16,5,[],0,0,0,36,341.75


for_qiangye_skill_79.json


,BOTTOM,LEFT,RIGHT,SKILL_77,TOP,WAIT,damage,平均伤害,总伤害,总伤害次数,总移动次数,SKILL_79,最终HP
monster_18714666,2,2,3,3,2,18,"[75.0, 75.0, 75.0]",75.0,225.0,3,9,NaN,NaN
hero_5002,0,1,14,NaN,5,9,[],0,0,0,20,4,775.0


In [359]:
def parse_result_json_detail(log_path):
    res = json.load(open(log_path, "r"))['update']
    # 去除掉init_state部分
    static = {}
    for r in res:
        actions,states,tick=r['action'],r['state'],r['tick']
        for i in range(len(actions)):
            #if actions[i]['action_type']=='SKILL_99':
                print(actions[i],states[i])
log_files_path = os.getcwd() + "/test_data"
    # 读取这个路径下所有文件
log_path = log_files_path + "/for_qiangye_skill_"+str(99)+".json"
parse_result_json_detail(log_path)

{'action_type': 'TOP', 'move_position': [5, 1, 4], 'id': 18714666, 'class': 'monster'} [['change', ['map', 0, 32, 'Block'], [2, 1]], ['change', ['map', 0, 33, 'Block'], [1, 2]], ['change', ['monster', 18714666, 'position', 2], [3, 4]]]
{'action_type': 'LEFT', 'move_position': [4, 1, 4], 'id': 18714666, 'class': 'monster'} [['change', ['map', 0, 16, 'Block'], [1, 2]], ['change', ['map', 0, 33, 'Block'], [2, 1]], ['change', ['monster', 18714666, 'position', 0], [5, 4]]]
{'action_type': 'SKILL_77', 'atk_range': [[4, 1, 3]], 'atk_position': [4, 1, 3], 'release_range': [[3, 1, 4], [4, 1, 3], [4, 1, 5], [5, 1, 4], [4, 1, 4]], 'damage': [[['hero', 5002], [94.5], [94.5], ['Equal']]], 'id': 18714666, 'class': 'monster'} [['change', ['hero', 5002, 'Hp'], [1000, 905.5]]]
{'action_type': 'WAIT', 'id': 5002, 'class': 'hero'} []
{'action_type': 'RIGHT', 'move_position': [5, 1, 4], 'id': 18714666, 'class': 'monster'} [['change', ['map', 0, 16, 'Block'], [2, 1]], ['change', ['map', 0, 33, 'Block'], [1

In [375]:
def parse_decision_log(log_path):
    res=json.load(open(log_path,"r"))
    for k,v in res.items():
        if 'stepname' in v:
            if 'evaluate' in v['stepname'] or '决策树' in v['stepname']:
                if v['stepname']=='evaluate执行判断' or v['stepname']=='evaluate执行动作' or v['stepname']=='决策树-首次查找最终选择' or  v['stepname']=='决策树-多次查找最终选择':
                   #if '判断是否满足恢复判定' in v['selection'] and v['result_len']=='True':
                        print(k,v)

In [376]:

tmp_log_file = os.getcwd()+"/../log/logs.json"
parse_decision_log(tmp_log_file)

2024-09-25 16:05:37.705726_6 {'stepname': 'evaluate执行判断', 'selection': '判断是否满足恢复判定', 'result_len': 'False'}
2024-09-25 16:05:37.706200_7 {'stepname': 'evaluate执行判断', 'selection': '判断是否满足逃跑条件', 'result_len': 'False'}
2024-09-25 16:05:37.706771_8 {'stepname': 'evaluate执行判断', 'selection': '判断是否满足逃跑条件', 'result_len': 'True'}
2024-09-25 16:05:37.707133_9 {'stepname': 'evaluate执行判断', 'selection': '判断警戒范围内是否有敌人', 'result_len': 'False'}
2024-09-25 16:05:37.708115_10 {'stepname': 'evaluate执行判断', 'selection': '判断是否有友军在战斗', 'result_len': 'False'}
2024-09-25 16:05:37.708542_11 {'stepname': 'evaluate执行判断', 'selection': '判断是否有boss', 'result_len': 'False'}
2024-09-25 16:05:37.711983_13 {'stepname': 'evaluate执行动作', 'action': '等待', 'result_len': 1}
2024-09-25 16:05:37.714383_14 {'stepname': '决策树-首次查找最终选择', 'result_len': 1, 'action': '等待'}
2024-09-25 16:05:38.110639_22 {'stepname': 'evaluate执行判断', 'selection': '判断是否满足恢复判定', 'result_len': 'False'}
2024-09-25 16:05:38.112417_23 {'stepname': 'evaluate执行判断'